<a href="https://colab.research.google.com/github/aniketjivani/generative_experiments/blob/master/generate_subs_from_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai-whisper pydub torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=642bd691f62eb21f374d8925be1d145e3317531900ac2f6e63918694d633c197
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [2]:
# add google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# Python code to convert video to audio
import moviepy.editor as mp
from datetime import datetime, timedelta
from pydub import AudioSegment
import os
from rich.progress import track
# Insert local audio file path
# audioclip = mp.AudioFileClip(r"/content/drive/MyDrive/audio_transcribe/audio_sciml_lecture.m4a")

audioclip = AudioSegment.from_file("/content/drive/MyDrive/audio_transcribe/audio_sciml_lecture.m4a")

shortclip = audioclip[:10000]

# clip for the first three minutes.
longclip = audioclip[:180000]

# # Insert Local Video File Path
# clip = mp.VideoFileClip(r"Video File")

# # Insert Local Audio File Path
# clip.audio.write_audiofile(r"Audio File")

In [16]:
# # play audioclip in Colab
# from IPython.display import Audio
# Audio(shortclip, rate=22050)

In [18]:
# Function to split large audio into chunks
def split_audio(input_audio_path, chunk_length_ms=30000):
    audio = AudioSegment.from_file(input_audio_path)
    duration_ms = len(audio)
    chunks = []

    for start_ms in range(0, duration_ms, chunk_length_ms):
        end_ms = min(start_ms + chunk_length_ms, duration_ms)
        chunk = audio[start_ms:end_ms]
        chunks.append(chunk)

    return chunks

# Function to transcribe and translate audio using Whisper
def transcribe_and_translate(audio_chunk, model):
    # Save the chunk as a temporary file
    temp_file = "/content/drive/MyDrive/audio_transcribe/temp_chunk.wav"
    audio_chunk.export(temp_file, format="wav")

    # Transcribe and translate to English with timestamps
    result = model.transcribe(temp_file, task="translate")

    # Remove temporary file
    os.remove(temp_file)

    return result

# Function to convert transcriptions to subtitle format
def generate_srt(transcription_results, output_srt_path):
    with open(output_srt_path, 'w') as f:
        index = 1
        for segment in transcription_results:
            start_time = segment['start']
            end_time = segment['end']
            text = segment['text']

            # Convert start and end times to the format HH:MM:SS,MS
            start_time_str = str(timedelta(seconds=start_time))
            end_time_str = str(timedelta(seconds=end_time))

            # Ensure formatting is consistent
            start_time_str = start_time_str[:-3]  # Remove excess microseconds
            end_time_str = end_time_str[:-3]

            f.write(f"{index}\n")
            f.write(f"{start_time_str} --> {end_time_str}\n")
            f.write(f"{text}\n\n")
            index += 1

# Main pipeline
def process_audio_to_subtitles(input_audio_path, output_srt_path):
    chunks = split_audio(input_audio_path, chunk_length_ms=30000)  # Split audio into 30s chunks
    transcription_results = []

    # Process each chunk
    for chunk in track(chunks, description="Processing 30s chunks one by one"):
        result = transcribe_and_translate(chunk, model)
        transcription_results.extend(result['segments'])  # Add each segment from the result

    # Generate subtitle file from transcription results
    generate_srt(transcription_results, output_srt_path)
    print(f"Subtitle file saved to {output_srt_path}")

In [19]:
import whisper
from pydub import AudioSegment
import os

# Initialize Whisper model
model = whisper.load_model("base")  # You can use "small", "medium", or "large" depending on your needs

# Example usage
input_audio_path = "/content/drive/MyDrive/audio_transcribe/audio_sciml_lecture.m4a"
output_srt_path = "/content/drive/MyDrive/audio_transcribe/output_subs.srt"

process_audio_to_subtitles(input_audio_path, output_srt_path)

  checkpoint = torch.load(fp, map_location=device)



Output()

Subtitle file saved to /content/drive/MyDrive/audio_transcribe/output_subs.srt
